In [127]:
import sys
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Dropout,Conv1D,MaxPooling1D,initializers,Input
from keras.models import Model
from keras import optimizers,regularizers
from keras.callbacks import History,EarlyStopping
from keras.optimizers import SGD
import keras_metrics
from sklearn import metrics

In [165]:
X=np.loadtxt('newdata.txt')
Y=np.loadtxt('newlabels.txt')
Y=Y.astype(int)

In [129]:
print(X.shape)
print(Y.shape)
X,Y= shuffle(X,Y)

(2600, 79)
(2600,)


In [130]:
n_values = np.max(Y) + 1
newY=np.eye(n_values)[Y]
print(newY.shape)

(2600, 7)


In [131]:
n=int(X.shape[0]*0.80)
length=X.shape[0]
X_train=X[0:n,:]
X_test=X[n:,:]

Y_train=Y[0:n]
Y_test=Y[n:]

newY_train=newY[0:n]
newY_test=newY[n:]

In [132]:
print(X_train.shape)
print(X_test.shape)

(2080, 79)
(520, 79)


In [133]:
print(Y_train.shape)
print(Y_test.shape)

print(newY_train.shape)
print(newY_test.shape)

(2080,)
(520,)
(2080, 7)
(520, 7)


In [162]:
seed = 7
np.random.seed(seed)
StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
accuracy=[]
f1=[]
precision=[]
recall=[]

f1_class=np.empty((0,7))
precision_class=np.empty((0,7))
recall_class=np.empty((0,7))
for train, test in kfold.split(X_train, newY_train,Y_train):
  # create model
    model = Sequential()
    model.add(Dense(32, input_dim=79, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(7, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
    
    model.fit(X_train[train], newY_train[train], epochs=10, batch_size=10, verbose=2)
    y_pred=model.predict(X_train[test])
    predict_class = np.argmax(y_pred, axis=1)
    
    #f1 values
    f1_value=metrics.f1_score(predict_class,Y_train[test],average='macro')
    f1_class_value=metrics.f1_score(predict_class,Y_train[test],average=None).reshape(1, -1)
    
    #precision values
    precision_value=metrics.precision_score(predict_class,Y_train[test],average='macro')
    precision_class_value=metrics.precision_score(predict_class,Y_train[test],average=None).reshape(1, -1)
    
    #recall values
    recall_value=metrics.recall_score(predict_class,Y_train[test],average='macro')
    recall_class_value=metrics.recall_score(predict_class,Y_train[test],average=None).reshape(1, -1)
    
    scores = model.evaluate(X_train[test], newY_train[test], verbose=0)
    accuracy.append(scores[1])
    
    f1.append(f1_value)
    precision.append(precision_value)
    recall.append(recall_value)
    
    f1_class=np.concatenate((f1_class,f1_class_value),axis=0)
    print(f1_class)
    precision_class=np.concatenate((precision_class,precision_class_value),axis=0)
    recall_class=np.concatenate((recall_class,recall_class_value),axis=0)
    
average_f1=f1_class.mean(axis=0)
average_precision=precision_class.mean(axis=0)
average_recall=recall_class.mean(axis=0)

Epoch 1/10
 - 8s - loss: 1.3560 - acc: 0.5439
Epoch 2/10
 - 1s - loss: 0.6071 - acc: 0.7819
Epoch 3/10
 - 1s - loss: 0.2928 - acc: 0.8882
Epoch 4/10
 - 1s - loss: 0.1802 - acc: 0.9549
Epoch 5/10
 - 1s - loss: 0.1283 - acc: 0.9681
Epoch 6/10
 - 1s - loss: 0.0935 - acc: 0.9778
Epoch 7/10
 - 1s - loss: 0.0713 - acc: 0.9874
Epoch 8/10
 - 1s - loss: 0.0518 - acc: 0.9922
Epoch 9/10
 - 1s - loss: 0.0417 - acc: 0.9922
Epoch 10/10
 - 1s - loss: 0.0342 - acc: 0.9952
[[0.98461538 1.         0.99047619 1.         1.         1.
  0.97196262]]
Epoch 1/10
 - 8s - loss: 1.4651 - acc: 0.5889
Epoch 2/10
 - 1s - loss: 0.5196 - acc: 0.8149
Epoch 3/10
 - 1s - loss: 0.2915 - acc: 0.8600
Epoch 4/10
 - 1s - loss: 0.2095 - acc: 0.8804
Epoch 5/10
 - 1s - loss: 0.1568 - acc: 0.9429
Epoch 6/10
 - 1s - loss: 0.1123 - acc: 0.9814
Epoch 7/10
 - 1s - loss: 0.0753 - acc: 0.9838
Epoch 8/10
 - 1s - loss: 0.0515 - acc: 0.9898
Epoch 9/10
 - 1s - loss: 0.0363 - acc: 0.9934
Epoch 10/10
 - 1s - loss: 0.0275 - acc: 0.9976
[[0

In [166]:
print("For training:")
print("Accuracy is :"+ str(np.mean(accuracy)))
print("F1 is :"+ str(np.mean(f1)))
print("Precision is: "+str(np.mean(precision)))
print("Recall is: "+str(np.mean(recall)))
print("\n")
print("Average F1 for all the classes is:" +str(average_f1))
print("Average F1 for all the classes is:" +str(average_precision))
print("Average F1 for all the classes is:" +str(average_recall))

For training:
Accuracy is :0.9817307692307692
F1 is :0.9839923647687894
Precision is: 0.9872366814158845
Recall is: 0.9813777799551258


Average F1 for all the classes is:[0.97194132 0.99103649 0.98813334 0.98346578 0.9830005  1.
 0.97036912]
Average F1 for all the classes is:[0.95952857 1.         0.99622642 0.97888954 0.97601224 1.
 1.        ]
Average F1 for all the classes is:[0.98477254 0.98238796 0.98042229 0.98849123 0.99090909 1.
 0.94266135]


In [ ]:
#Based on the other models, we will compare the score against the other models
#and then see how well the model will generalise based on the test scores.